## $LU$- factorization

Any non-singular matrix, denoted as "A," can be broken down into two special matrices: a lower triangular matrix "L" and an upper triangular matrix "U". 

This decomposition is based on Gaussian elimination and can be quite useful in practical computations, especially when solving linear systems: instead of solving "Ax = b" directly, we break it down into two simpler steps. First, we find "y" in the equation "Ly = b," where "L" is lower triangular. This can be solved efficiently with forward substitution. Then, we use "y" to find "x" in the equation "Ux = y," where "U" is upper triangular, which can be solved with back substitution. 

This two-step process reduces the time it takes to find "x" from a cubic (O(n^3)) operation to a quadratic (O(n^2)) operation, making it much faster for large systems. However, determining the "L" and "U" matrices still requires cubic operations, but this only needs to be done once for a given "A" matrix and can be reused to solve for "x" with different "b" vectors.

It's important to note that there are various ways to decompose "A" into "L" and "U," and these matrices are not unique. One common method is through Gaussian elimination, but others exist.


Let $A$ be a matrix. Then we can define $U$ to be its echelon form. So we can turn $A$ into $U$ using row operations. To find matrix $L$, we do the opposite of the operations we used to get to $U$ but on the identity matrix. For example, if we did this operation on $A$:
$$
E_i - m_{ji}E_j\rightarrow E_i
$$
To go back to $A$, we perform the opposite operation:
$$
E_i + m_{ji}E_j\rightarrow E_i
$$
If we do this on the identity matrix (which has the same size as $A$), we get $\mathbf{L}$. This decomposition is not unique manily because the echelon form is not unique. 


### Example:
The following computations find $U$ for the matrix $A$:

$$
\mathbf{A} = \begin{bmatrix}1&1&0\\2&1&-1\\3&-1&-1\end{bmatrix}
\xrightarrow{E_2-2E_1}
\begin{bmatrix}1&1&0\\0&-1&-1\\3&-1&-1\end{bmatrix}
\xrightarrow{E_3-3E_1}
\begin{bmatrix}1&1&0\\0&-1&-1\\0&-4&-1\end{bmatrix}
\xrightarrow{E_3-4E_2}
\begin{bmatrix}1&1&0\\0&-1&-1\\0&0&3\end{bmatrix} = \mathbf{U}
$$

Now we do the reverse operations to identity matrix to find $L$

$$
\mathbf{I} = \begin{bmatrix}1&0&0\\0&1&0\\0&0&1\end{bmatrix}
\xrightarrow{E_3+4E_2}
\begin{bmatrix}1&0&0\\0&1&0\\0&4&1\end{bmatrix}
\xrightarrow{E_3+3E_1}
\begin{bmatrix}1&0&0\\0&1&0\\3&4&1\end{bmatrix}
\xrightarrow{E_2+2E_1}
\begin{bmatrix}1&0&0\\2&1&0\\3&4&1\end{bmatrix} = \mathbf{L}
$$


Use numpy to check our calculation:

In [4]:
# You code: 

### Python / NumPy implementation of a simple $\mathbf{LU}$ factorization

We can also use the simple impelementation:

In [5]:
def lu(A):
    
    #Get the number of rows
    n = A.shape[0]
    
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    
    #Loop over rows
    for i in range(n):
            
        #Eliminate entries below i with row operations 
        #on U and reverse the row operations to 
        #manipulate L
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]
        
    return L, U

Lets use this function to check our calculation in first example:

In [ ]:
# you code

If we encounter a zero in a specific position (e.g., $a_{ii} = 0$) while solving equations, it can mess our algorithm. To fix this, we swap rows to make it work correctly. Specifically, we pick rows with zeros on the main diagonal and exchange them with the first row below that contains a nonzero number in that column.

To keep a record of these row swaps, we create a permutation matrix denoted as $\mathbf{P}$. This matrix becomes valuable when we solve the equations in a different order. It's akin to rearranging puzzle pieces for a simpler solution, and $\mathbf{P}$ documents these changes. As a result, we obtain a matrix with exactly one nonzero entry in each row and column, and each nonzero entry is 1.

__Theorem: For every invertible matrix $A,"$ there exists a decomposition $PA = LU,$ where $P$ represents the permutation matrix.__

In [9]:
def plu(A):
    
    #Get the number of rows
    n = A.shape[0]
    
    #Allocate space for P, L, and U
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    P = np.eye(n, dtype=np.double)
    
    #Loop over rows
    for i in range(n):
        
        #Permute rows if needed
        for k in range(i, n): 
            if ~np.isclose(U[i, i], 0.0):
                break
            U[[k, k+1]] = U[[k+1, k]]
            P[[k, k+1]] = P[[k+1, k]]
            
        #Eliminate entries below i with row 
        #operations on U and #reverse the row 
        #operations to manipulate L
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]
        
    return P, L, U

### Application: 

As discussed in class, once we have $\mathbf{L}$ and $\mathbf{U}$ we can solve for $A\vec{x} = \vec{b}$ for as many right-hand side vectors $\vec{b}$ as desired very quickly using the following two step process. First we let $\vec{y}=\mathbf{U}\vec{x}$ and then solve for $\mathbf{L}\vec{y}=\vec{b}$ for $\vec{y}$ by using forward substitution. The pseudocode for this is as follows:

In [11]:
def forward_substitution(L, b):
    
    #Get number of rows
    n = L.shape[0]
    
    #Allocating space for the solution vector
    y = np.zeros_like(b, dtype=np.double);
    
    #Here we perform the forward-substitution.  
    #Initializing  with the first row.
    y[0] = b[0] / L[0, 0]
    
    #Looping over rows in reverse (from the bottom  up),
    #starting with the second to last row, because  the 
    #last row solve was completed in the last step.
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y

### Example: For the above matrix $A$, choose a vector $\vec{b}$ and cfind the solution of $A\vec{x} = \vec{b}$

In [8]:
# you code

## Diagonalization

An $n \times n$ matrix $A$ is called __diagonalizable__ if it is _similar_ to a diagonal matrix. More precisely, there exists an invertible $n \times n$ matrix $P$ and an $n \times n$ diagonal matrix $D$ such that $A = PDP^{-1}$.

It turns out an $n\times n$ matrix is diagonalizable if we can form a basis $\mathbb{R}^n$ using eigenvectors of $A$:

__Theorem 1__

An $n \times n$ matrix $A$ is diagonalizable if and only if $A$ has $n$ linearly independent eigenvectors.


In fact, suppose $\lambda_1, \lambda_2, \dots, \lambda_n$ are eigenvalues of $A$, and $\{\vec{v_1}, \vec{v}_2, \dots, \vec{v}_n\}$ is the linearly independent set of eigenvectors of $A$. If we form the matrix $P$ by taking the eigenvectors as columns, and $D$ is a diagonal matrix with eigenvalues $\lambda_1, \lambda_2, \dots, \lambda_n$ on the diagonal, then we have $A = PDP^{-1}$.


Finding eigenvectors in general is not an easy task, and is not clear how to check which $n \times n$ matrices have $n$ linearly independent eigenvectors. The next theorem partially addresses this issue.


__Theorem 2__

The eigenvectors corresponding to distinct eigenvalues are linearly independent.

Combining Theorem 1 and Theorem 2, we obtain the following corollary:

__Corollary 1__

An $n \times n$ matrix with $n$ distinct eigenvalues is diagonalizable.



__Example 1__  Is
$
A = \begin{bmatrix} 1 & 3 & 2 \\ -3 & -5 & 3 \\ 3 & 3 & 1 \end{bmatrix}
$
diagonalizable? If yes find a diagonalization for it. 

In [21]:
# you code

### Diagonalization as a change of basis 

Recall that similar matrices represent the same linear map under two different choices of a pair of bases for $\mathbb{R}^n$ and $\mathbb{R}^m$. More precisely, suppose $A = PCP^{-1}$ ($C$ is not necessarily diagonal), then the matrix representation of $T$ with respect to the basis $B$ (formed by the columns of $P$) is $C$. Conversely, changing the basis of $\mathbb{R}^n$ leads to a matrix representation of $T$ that is similar to $A$ (section 3.4). This particularly applies to diagonalizable matrices because a matrix is diagonalizable if it is similar to a diagonal matrix.

__Theorem 2__ (Diagonal Matrix Representation)

Suppose $T: \mathbb{R}^n \to \mathbb{R}^n$ and $A$ is the standard matrix representation of $T$ that is diagonalizable. Let $A = PDP^{-1}$ be a diagonalization of $A$ where $D$ is an $n\times n$ diagonal matrix, and $P$ is an invertible $n\times n$ matrix. If $B$ is the basis for $\mathbb{R}^n$ formed by the columns of $P$, then $D$ is the matrix representation of the transformation $T: (\mathbb{R}^n, B) \to (\mathbb{R}^n, B)$.

__Example 3__ Let $T: \mathbb{R}^2 \to \mathbb{R}^2$ and $A = \begin{bmatrix} 7 & 2 \\ -4 & 2 \end{bmatrix}$ be the standard matrix representation of $T$. Find a basis $B$ for $T$ such that the matrix representation of $T$ with respect to $B$ is diagonal.

__Solution:__

From Example 1, we know that $A = PAP^{-1}$ where $P = \begin{bmatrix} 1 & 1 \\ -1 & -2 \end{bmatrix}$ and $D = \begin{bmatrix} 5 & 0 \\ 0 & 3 \end{bmatrix}$.

The columns of $P$ are eigenvectors of $A$, and they form a basis $B$ for $\mathbb{R}^2$. By Theorem 2, $D$ is the matrix representation of $T$ with respect to $B$. In fact, the mappings $\vec{x} \to A\vec{x}$ and $\vec{x} \to D\vec{x}$ describe the same linear transformation, relative to different bases.


Excercises

1. Suppose $C=\begin{bmatrix}
3 & 0 & 1 & 0 & 0\\
0 & 3 & 0 & 0 & 0\\
0 & 0 & 3 & 0 & 0\\
0 & 0 & 3 & 2 & 0\\
0 & 0 & 0 & 0 & 2
\end{bmatrix}$. Is $C$ diagonalizable?


2. Let $A$ be a $5\times 5$ matrix, with two eigenvalues. One eigenspace is three-dimensional, the other is two dimensional. Is $A$ diagonalizable?


3. Compute $A^{200}$ where $A = \begin{bmatrix} 4 & -3\\ 2 & -1 \end{bmatrix}$. 


# References:

1. S. Anbouhi Python for linear Algebra". Available at: https://timothyprojectgig.github.io/JB_Math_Textbook/Undergrad/Linear/0.html

2. G. Strang, "Linear Algebra and Learning from Data," 2019.

3. J. Foster, "Numerical Methods". Available at: https://johnfoster.pge.utexas.edu/numerical-methods-book/PythonIntro